In [1]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# train and test 
train_dir = 'E:/Cancer_Dataset/train'
test_dir = 'E:/Cancer_Dataset/test'

# Check if directories exist and print the number of files
print("Training data directory:", os.path.isdir(train_dir))
print("Test data directory:", os.path.isdir(test_dir))

# Initialize ImageDataGenerators with preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create train and test generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Check the number of images found in train and test directories
print("Found", train_generator.samples, "training images.")
print("Found", test_generator.samples, "test images.")

# Print the structure of a single batch of data (for debugging)
try:
    train_batch = next(train_generator)
    print("Train batch shape:", train_batch[0].shape)  # Should print (batch_size, 150, 150, 3)
    test_batch = next(test_generator)
    print("Test batch shape:", test_batch[0].shape)  # Should print (batch_size, 150, 150, 3)
except Exception as e:
    print("Error while loading batches:", str(e))

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

# Save the model after training
model.save('cancer_cell_detection_model.h5')

# Plot the training and validation accuracy/loss over epochs
# Accuracy plot
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


def plot_sample_images(generator, num_images=5):
    images, labels = next(generator) 
    plt.figure(figsize=(12, 12))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(images[i])
        label = 'Cancer' if labels[i] == 1 else 'Not Cancer'  
        plt.title(f'Label: {label}')
        plt.axis('off')
    plt.show()

plot_sample_images(train_generator, num_images=5)


plot_sample_images(test_generator, num_images=5)